In [1]:
import numpy as np
import cs_vqe as c
import ast
import os

from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp

In [2]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [3]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [4]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
mol_key = 'H3_STO-3G_singlet_1+'  
# mol_key = 'H1-He1_3-21G_singlet_1+'
# mol_key = 'H3_3-21G_singlet_1+'
# mol_key ='H2-O1_STO-3G_singlet'


# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'III': -1.7512307459285525,
 'IIX': -0.023568152336180023,
 'IIZ': -0.017109477140260287,
 'IXI': 0.02356815233618002,
 'IXZ': 0.02356815233617983,
 'IYY': -0.07195737217001562,
 'IZI': -0.017109477140260287,
 'IZX': -0.023568152336179825,
 'IZZ': 0.31270210682950855,
 'XII': 0.01872992170537467,
 'XIX': 0.023568139980123585,
 'XIZ': 0.01872992170537467,
 'XXI': -0.023568139980123585,
 'XXX': 0.03597868636603963,
 'XXZ': -0.023568139980123585,
 'XYY': -0.03597868636603963,
 'XZI': 0.01872992170537467,
 'XZX': 0.023568139980123585,
 'XZZ': 0.01872992170537467,
 'YIY': 0.023568139980123585,
 'YXY': 0.03597868636603963,
 'YYI': -0.023568139980123585,
 'YYX': 0.03597868636603963,
 'YYZ': -0.023568139980123585,
 'YZY': 0.023568139980123585,
 'ZII': -0.45436486525596403,
 'ZIX': -0.023568152336180023,
 'ZIZ': 0.37110605476609787,
 'ZXI': 0.02356815233618002,
 'ZXZ': 0.02356815233617983,
 'ZYY': -0.07195737217001562,
 'ZZI': 0.37110605476609804,
 'ZZX': -0.023568152336179825,
 'ZZZ': -0.2878

In [7]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  3


# Get non-contextual H

In [8]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [9]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [10]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [11]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [12]:
Z_list

['III', 'IZZ', 'ZIZ', 'ZZI']

In [13]:
T_list

['IIZ', 'IZI', 'XXX', 'XYY', 'YXY', 'YYX', 'ZII', 'ZZZ']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [14]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [15]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['III', 'IZZ', 'ZIZ', 'ZZI']
G (independent) Z list: ['ZIZ', 'IZZ']


In [16]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIZ', 'XXX']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [17]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'III': [[], [], 1],
 'IIZ': [[], ['IIZ'], 1],
 'IZI': [['IZZ'], ['IIZ'], 1],
 'IZZ': [['IZZ'], [], 1],
 'XXX': [[], ['XXX'], 1],
 'XYY': [['IZZ'], ['XXX'], (-1+0j)],
 'YXY': [['ZIZ'], ['XXX'], (-1+0j)],
 'YYX': [['ZIZ', 'IZZ'], ['XXX'], (-1+0j)],
 'ZII': [['ZIZ'], ['IIZ'], 1],
 'ZIZ': [['ZIZ'], [], 1],
 'ZZI': [['ZIZ', 'IZZ'], [], 1],
 'ZZZ': [['ZIZ', 'IZZ'], ['IIZ'], 1]}

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [18]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [19]:
fn_form

[2,
 2,
 [[-1.7512307459285525, [], [], 'III'],
  [-0.017109477140260287, [], [0], 'IIZ'],
  [-0.017109477140260287, [1], [0], 'IZI'],
  [0.31270210682950855, [1], [], 'IZZ'],
  [0.03597868636603963, [], [1], 'XXX'],
  [(0.03597868636603963-0j), [1], [1], 'XYY'],
  [(-0.03597868636603963+0j), [0], [1], 'YXY'],
  [(-0.03597868636603963+0j), [0, 1], [1], 'YYX'],
  [-0.45436486525596403, [0], [0], 'ZII'],
  [0.37110605476609787, [0], [], 'ZIZ'],
  [0.37110605476609804, [0, 1], [], 'ZZI'],
  [-0.2878474382772282, [0, 1], [0], 'ZZZ']]]

In [20]:
Energy_function = c.energy_function(fn_form)

In [21]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [22]:
 Energy_function(*q_variables,*r_variables)

-1.897415425779325

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [23]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-2.903212918716725
[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]


In [24]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [25]:
ground_state_params

[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [140]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIZ', 'IZZ']
['IIZ', 'XXX']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [138]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

In [139]:
# rotations to map A to single Pauli operator!
Rotations_list

[[3.342131729291596, 'XXY'],
 ['pi/2', 'YII'],
 ['pi/2', 'YIZ'],
 ['pi/2', 'IYI'],
 ['pi/2', 'IYZ']]

In [29]:
# rotations to diagonlize G
diagonalized_generators_GuA

['ZII', 'IZI', 'IIZ']

In [30]:
eigen_vals_nonC_ground_state_GuA_ops

array([ 1., -1.,  1.])

# NEW LCU method

In [131]:
N_index=0
check_reduction=True
N_Qubits= hamiltonians[mol_key][1]
R_LCU, Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops= c.diagonalize_epistemic_LCU(
                                                                                  model,
                                                                                  fn_form,
                                                                                  ground_state_params,
                                                                                  N_Qubits,
                                                                                  N_index,
                                                                                  check_reduction=check_reduction)

# Restricting the Hamiltonian to a contextualsubspace

(Section B of https://arxiv.org/pdf/2011.10027.pdf)

In the rotated basis the Hamiltonian is restricted to the subspace stabilized by the noncontextual generators $G_{j}'$

In [44]:
print(diagonalized_generators_GuA) # G_j' terms!

['ZII', 'IZI', 'IIZ']


The quantum correction is then obtained by minimizing the expectation value of this resticted Hamiltonian!

(over +1 eigenvectors of the remaining non-contextual generators $\mathcal{A}'$)

In [45]:
print(Ci1_list) # mathcal(A)

['IIZ', 'XXX']


- $\mathcal{H}_{1}$ denotes Hilbert space of $n_{1}$ qubits acted on by by the single qubit $G_{j}'$ terms
- $\mathcal{H}_{2}$ denotes Hilbert space of remaining $n_{2}$

Overall full Hilbert space is: $\mathcal{H}=\mathcal{H}_{1} \otimes \mathcal{H}_{2}$

The **contextual Hamiltonian** in this rotated basis is:

$$H_{c}'=\sum_{P \in \mathcal{S_{c}'}} h_{P}P$$

The set of Pauli terms in $H_{c}'$ is $\mathcal{S_{c}'}$, where terms in $\mathcal{S_{c}'}$ act on both $\mathcal{H}_{1}$ and $\mathcal{H}_{2}$ subspaces in general!

We can write $P$ terms as:

$$P=P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}}$$

$P$ commutes with an element of $G'$ if and only if $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ does

As the generators $G'$ act only on $\mathcal{H}_{1}$

If $P$ anticommutes with any element of $G'$ then its expection value in the noncontextual state is zero

Thus any $P$ must commute with all elements of $G'$ and so $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ too

As the elements of $G'$ are single-qubit Pauli $Z$ operators acting in $\mathcal{H}_{1}$:

In [46]:
print(diagonalized_generators_GuA) # G_j' terms!

['ZII', 'IZI', 'IIZ']


$P_{1}$ must be a product of such operators!

**As the exepcation value of $P_{1}$ is some $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [47]:
eigen_vals_nonC_ground_state_GuA_ops

array([ 1., -1.,  1.])

Let $|\psi_{(\vec{q}, \vec{r})} \rangle$ be any quantum state consistent with the nonconxtual ground state $(\vec{q}, \vec{r})$... aka gives correct expection values of:

In [48]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

['ZII', 'IZI', 'IIZ']
[ 1. -1.  1.]


Then the action of any $P$ which allows our contextual correction has the form:

$$P |\psi_{(\vec{q}, \vec{r})} \rangle = \big( P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

$$ = p_{1}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

- repeating above, but $p_{1}$ is the expectation value of $P_{1}$ determiend by the noncontextual ground state!

Thus we can denote $H_{c}' |_{(\vec{q}, \vec{r})}$ as the restriction of $H_{c}'$ on its action on the noncontextual ground state $(\vec{q}, \vec{r})$:

$$H_{c}' |_{(\vec{q}, \vec{r})} =\sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) $$

$$=\mathcal{I}_{\mathcal{H}_{1}} \otimes H_{c}'|_{\mathcal{H}_{2}} $$


where we can write:
$$H_{c}'|_{\mathcal{H}_{2}} = \sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}P_{2}^{\mathcal{H}_{2}}$$



Cleary this Hamiltonian on $n_{2}$ qubits is given by:

$$n_{2} = n - |G|$$

- $|G|=$ number of noncontextual generators $G_{j}$

In [49]:
from copy import deepcopy
import pprint 

```quantum_correction``` function

In [79]:
new = R_OPER*Ham_openF


50

In [108]:
R_LCU

[0.9238795325112867 [], 0.3826834323650898j [X0 X1 Y2]]

In [133]:
n_q = len(diagonalized_generators_GuA[0])

# rotated_H = deepcopy(ham)  ##<-- full Hamiltonian

# iteratively perform R rotation over all terms in orginal Hamiltonian


### LCU rotation
if R_LCU is not None:
    Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(ham)  ##<-- full Hamiltonian
    R_openF = QubitOperator()
    for op in R_LCU:
        R_openF+=op
    rotated_H = R_openF*R_openF

            
rotated_H = conv_scr.Openfermion_to_dict(rotated_H, n_q)

for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_H.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_H[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_H[P]
                
    rotated_H = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_H

{'III': (0.7071067811865475+0j), 'ZZY': 0.7071067811865476j}

next find where Z indices in $G'$

In [134]:
z_indices = []
for d in diagonalized_generators_GuA:
    for i in range(n_q):
        if d[i] == 'Z':
            z_indices.append(i)
            
print(diagonalized_generators_GuA)
print(z_indices)

['ZII', 'IZI', 'IIZ']
[0, 1, 2]


**The exepcation value of $P_{1}$ terms are $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [135]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

['ZII', 'IZI', 'IIZ']
[ 1. -1.  1.]


We need to ENFORCE the diagnal geneators assigned values in the diagonal basis to these expectation values above^^^

In [136]:
ham_red = {}
    
for P in rotated_H.keys():
    sgn = 1
    for j, z_index in enumerate(z_indices): # enforce diagonal generator's assigned values in diagonal basis
        if P[z_index] == 'Z':
            sgn = sgn*eigen_vals_nonC_ground_state_GuA_ops[j] #<- eigenvalue of nonC ground state!
        elif P[z_index] != 'I':
            sgn = 0

    if sgn != 0:
        # construct term in reduced Hilbert space
        P_red = ''
        for i in range(n_q):
            if not i in z_indices:
                P_red = P_red + P[i]
        if P_red in ham_red.keys():
            ham_red[P_red] = ham_red[P_red] + rotated_H[P]*sgn
        else:
            ham_red[P_red] = rotated_H[P]*sgn
            
ham_red

{'': (0.7071067811865475+0j)}

In [66]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

-2.903212918716725

In [ ]:
c.quantum_correction(nonCon_H,model,fn_form,ground_state_params)

In [ ]:
c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           list(range(hamiltonians[mol_key][1])))[-1] == rotated_H ### aka when considering all qubit problem it is equal to rotated H!

For some reason it seems that when considering full Hamiltonian there is no reduction in the number of terms!

Q. Do you expect any term reduction when doing CS-VQE?

In [ ]:
n2 = hamiltonians[mol_key][1]-len(diagonalized_generators_GuA)
n2

In [ ]:
ham_red

In [ ]:
ham==Con_H

In [ ]:
n_q = len(diagonalized_generators_GuA[0])
rotated_Hcon = deepcopy(Con_H)

# iteratively perform R rotation over all terms in orginal Hamiltonian
for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_Hcon.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_Hcon[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_Hcon[P]
                
    rotated_Hcon = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_Hcon

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

In [ ]:
new={}
for P1_P2 in rotated_Hcon.keys():
    Z_indices = [i for i, sigma in enumerate(P1_P2) if sigma=='Z']
    
    I1_P2=list(deepcopy(P1_P2))
    sign=1
    for ind in Z_indices:
        sign*=p1_dict[ind]
        I1_P2[ind]='I'
    I1_P2=''.join(I1_P2)
    
    new[I1_P2]=rotated_Hcon[P1_P2]*sign
    
new    

In [ ]:
len(rotated_Hcon)-len(diagonalized_generators_GuA)
# len(new)

In [ ]:
H = conv_scr.Get_Operfermion_Hamiltonian(new)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
H_reduced_subspace={}
for P in rotated_H.keys():
    
    sign=1
    for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
        Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
        if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
            
            sign*=eigen_val #<- eigenvalue of nonC ground state!
            
        elif P[Z_index]!= 'I': 
            sign=0 # MUST anti-commute!
    
    # build reduced Hilbert Space
    if sign!=0:
        P_new = list(deepcopy(P))
        P_new[Z_index]='I'
        P_new= ''.join(P_new)
        
        if P_new in H_reduced_subspace.keys():
            H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_H[P]*sign
        else:
            H_reduced_subspace[P_new] = rotated_H[P]*sign
            
#     else:
#         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
print(len(rotated_H))
print(len(H_reduced_subspace))

In [ ]:
# H_reduced_subspace

In [ ]:
lowest_eigenvalue

In [ ]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp
H = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
lowest_eigenvalue

In [ ]:
Hfull = conv_scr.Get_Operfermion_Hamiltonian(ham)
sparseHfull = qubit_operator_sparse(Hfull, n_qubits=hamiltonians[mol_key][1])
FCI = sp.sparse.linalg.eigsh(sparseHfull, which='SA', k=1)[0][0]
print('FCI=', FCI)

In [ ]:
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

In [ ]:
H_reduced_subspace={}
for P in rotated_Hcon.keys():
    
    new_sign=1
    P_new = list(P)
    for index, sigma in enumerate(P):
        if sigma == 'Z':
            new_sign*=p1_dict[index]
            P_new[index]='I'
    
    P_new = ''.join(P_new)
    H_reduced_subspace[P_new] = rotated_Hcon[P]*new_sign

In [ ]:
H_reduced_subspace

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

In [ ]:
# H_reduced_subspace={}
# for P in rotated_Hcon.keys():
    
#     p1=1
#     for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
#         Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
#         if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
#             p1*=eigen_val #<- eigenvalue of nonC ground state!
            
#             P1_P2 = list(deepcopy(P))
#             P1_P2[Z_index]='I'
#             I1_P2= ''.join(P1_P2)
            
#             if I1_P2 in H_reduced_subspace.keys():
#                 H_reduced_subspace[I1_P2] += rotated_Hcon[P]*p1
#             else:
#                 H_reduced_subspace[I1_P2] = rotated_Hcon[P]*p1
            
#         elif P[Z_index]== 'I':
#             H_reduced_subspace[P] = rotated_Hcon[P]
            
#         elif P[Z_index]!= 'I': 
#             sign=0 # MUST anti-commute!
#             H_reduced_subspace[P]=0
    
# #         # build reduced Hilbert Space
# #         if sign!=0:
# #             if P_new in H_reduced_subspace.keys():
# #                 H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_Hcon[P]*sign
# #             else:
# #                 H_reduced_subspace[P_new] = rotated_Hcon[P_new]*sign
            
# # #     else:
# # #         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
H_reduced_subspace

In [ ]:
nonCon_Energy = lowest_eigenvalue

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
Con_Energy = sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

Con_Energy+nonCon_Energy

In [ ]:
FCI

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
FCI-lowest_eigenvalue

In [ ]:
Con_Energy

In [ ]:
c.commute(P_gen, P)

In [86]:
script_A = model[1]

qubit_script_A_list = [conv_scr.convert_op_str(op, 1) for op in script_A]
qubit_script_A_list

[1 [Z2], 1 [X0 X1 X2]]

In [88]:
import Unitary_partitioning_LCU_method as LCU_UP

In [91]:
N_index=0
N_Qubits=3
R_linear_comb_list, Pn, gamma_l  = LCU_UP.Get_R_op_list(qubit_script_A_list, 
                                                 N_index, 
                                                 N_Qubits, 
                                                 check_reduction=False, 
                                                 atol=1e-8,
                                                 rtol=1e-05)
Pn

1 [Z2]

In [100]:
original_A = QubitOperator()
for op in qubit_script_A_list:
    original_A+=op
original_A

1.0 [X0 X1 X2] +
1.0 [Z2]

In [101]:
R_test = QubitOperator()
for op in R_linear_comb_list:
    R_test+=op
R_test

0.9238795325112867 [] +
0.3826834323650898j [X0 X1 Y2]

In [103]:
R_test*original_A*R_test

(0.9999999999999999+0j) [X0 X1 X2] +
(1+0j) [Z2]

In [107]:
R_test*Pn*R_test

(1+0j) [Z2]

In [105]:
gamma_l

1.4142135623730951